In [439]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

In [440]:
#   load data
df = pd.read_csv("..\\data\\02_cleanData_properties_data.csv", index_col="ID")
df.shape

(18348, 40)

In [441]:
df.drop_duplicates(inplace=True)
df.shape

(18124, 40)

In [442]:
columns_to_take = [
    "type", "region", "province", "localityType", 
    "bedroomCount", "netHabitableSurface", "condition", "epcScore",
    "bathroomCount", "showerRoomCount", "toiletCount", 
    "hasLift", "hasGarden", "hasTerrace", 
    "gardenSurface", "terraceSurface",
    "fireplaceExists", "hasSwimmingPool", "hasAirConditioning", 
    "price" 
    ]
    # "district", "postalCode", 
    # "kitchen", "pricePerMeter", "constructionYear", 
    # "parkingCountIndoor", "parkingCountOutdoor", "parkingCountClosedBox", 
    # "primaryEnergyConsumptionPerSqm", "hasDoubleGlazing", 
df = df[columns_to_take]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18124 entries, 10616910 to 10616831
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   type                 18124 non-null  object 
 1   region               18124 non-null  object 
 2   province             18124 non-null  object 
 3   district             18124 non-null  object 
 4   postalCode           18124 non-null  int64  
 5   localityType         18124 non-null  int64  
 6   bedroomCount         18124 non-null  float64
 7   netHabitableSurface  16313 non-null  float64
 8   condition            13431 non-null  object 
 9   bathroomCount        15972 non-null  float64
 10  showerRoomCount      18124 non-null  float64
 11  toiletCount          13910 non-null  float64
 12  hasLift              12198 non-null  object 
 13  hasGarden            5713 non-null   object 
 14  hasTerrace           11535 non-null  object 
 15  gardenSurface        3533 non-n

In [443]:
df.isna().sum()

type                       0
region                     0
province                   0
district                   0
postalCode                 0
localityType               0
bedroomCount               0
netHabitableSurface     1811
condition               4693
bathroomCount           2152
showerRoomCount            0
toiletCount             4214
hasLift                 5926
hasGarden              12411
hasTerrace              6589
gardenSurface          14591
terraceSurface         10886
fireplaceExists            0
hasSwimmingPool         8746
hasAirConditioning     15982
price                      0
dtype: int64

In [444]:
df["typeNum"] = df["type"].apply(lambda x: 1 if x == "HOUSE" else 2)
df[["type", "typeNum"]].head()

,type,typeNum
ID,,
10616910,APARTMENT,2
10616880,APARTMENT,2
10616933,APARTMENT,2
10616917,APARTMENT,2
10616913,APARTMENT,2


In [445]:
df.groupby(["typeNum"])["typeNum"].count()

typeNum
1    9170
2    8954
Name: typeNum, dtype: int64

In [446]:
df["regionNum"] = None
df.loc[df["region"] == "Brussels", "regionNum"] = 1
df.loc[df["region"] == "Flanders", "regionNum"] = 2
df.loc[df["region"] == "Wallonie", "regionNum"] = 3

In [447]:
df["provinceNum"] = None
df.loc[df["province"] == "Brussels", "provinceNum"] = 1
df.loc[df["province"] == "Antwerp", "provinceNum"] = 2
df.loc[df["province"] == "East Flanders", "provinceNum"] = 3
df.loc[df["province"] == "Flemish Brabant", "provinceNum"] = 4
df.loc[df["province"] == "Limburg", "provinceNum"] = 5
df.loc[df["province"] == "West Flanders", "provinceNum"] = 6
df.loc[df["province"] == "Hainaut", "provinceNum"] = 7
df.loc[df["province"] == "Liège", "provinceNum"] = 8
df.loc[df["province"] == "Luxembourg", "provinceNum"] = 9
df.loc[df["province"] == "Namur", "provinceNum"] = 10
df.loc[df["province"] == "Walloon Brabant", "provinceNum"] = 11

In [448]:
df.groupby(["regionNum", "provinceNum"])["typeNum"].count()

regionNum  provinceNum
1          1              2506
2          2              3125
           3              2575
           4              1382
           5               749
           6              3166
3          7              1524
           8              1265
           9               409
           10              664
           11              759
Name: typeNum, dtype: int64

In [449]:
df.drop(df[df["bedroomCount"] > 12].index, inplace=True)
df.groupby("bedroomCount")["typeNum"].sum()

bedroomCount
1.0      4034
2.0     10823
3.0      7790
4.0      2739
5.0       955
6.0       388
7.0       136
8.0        82
9.0        21
10.0       35
11.0       13
12.0       22
Name: typeNum, dtype: int64

In [450]:
df.dropna(subset=['netHabitableSurface'], inplace=True)
df["netHabitableSurface"].isna().sum()

0

In [451]:
df["condition"].fillna(value="UNKNOWN", inplace=True)
df["conditionNum"] = None
df.loc[df["condition"] == "AS_NEW", "conditionNum"] = 1
df.loc[df["condition"] == "JUST_RENOVATED", "conditionNum"] = 2
df.loc[df["condition"] == "GOOD", "conditionNum"] = 3
df.loc[df["condition"] == "TO_BE_DONE_UP", "conditionNum"] = 4
df.loc[df["condition"] == "TO_RENOVATE", "conditionNum"] = 5
df.loc[df["condition"] == "TO_RESTORE", "conditionNum"] = 6
df.loc[df["condition"] == "UNKNOWN", "conditionNum"] = 0
df.groupby("conditionNum")["typeNum"].sum()

conditionNum
0    6330
1    6001
2    1476
3    7327
4    1751
5    1589
6      98
Name: typeNum, dtype: int64

In [452]:
df["bathroomCount"].fillna(value=0, inplace=True)
df.loc[df["showerRoomCount"] < 1, "showerRoomCount"] = 0
df.loc[df["showerRoomCount"] > 14, "showerRoomCount"] = 0
df["toiletCount"].fillna(value=0, inplace=True)

In [453]:
# df.groupby("bathroomCount")["type"].count()
# df.groupby("showerRoomCount")["type"].count()
# df.groupby("toiletCount")["type"].count()

In [454]:
binary_to_fix = ["hasLift", "hasGarden", "hasTerrace", "fireplaceExists", "hasSwimmingPool", "hasAirConditioning"]
for btf in binary_to_fix:
    df[btf].fillna(value=False, inplace=True)
    df[btf] = df[btf].apply(lambda x: 1 if x == True else 0)


In [455]:
df[["gardenSurface", "terraceSurface"]].isna().sum()

gardenSurface     13034
terraceSurface     9462
dtype: int64

In [456]:
df["gardenSurface"].fillna(value=0, inplace=True)
df["terraceSurface"].fillna(value=0, inplace=True)

In [457]:
df.groupby("hasAirConditioning")["type"].count()

hasAirConditioning
0    16147
1      145
Name: type, dtype: int64

In [458]:
new_order = [
    "type", "typeNum", "region", "regionNum", "province", "provinceNum", "district", "postalCode", 
    "localityType", "bedroomCount", "netHabitableSurface", "condition", "conditionNum",
    "bathroomCount", "showerRoomCount", "toiletCount", 
    "hasLift", "fireplaceExists", "hasSwimmingPool", "hasAirConditioning", 
    "hasGarden", "gardenSurface", "hasTerrace", "terraceSurface",
    "price"
    ]
df = df[new_order]

In [459]:
df.shape

(16292, 25)

In [460]:
df.isna().sum()

type                   0
typeNum                0
region                 0
regionNum              0
province               0
provinceNum            0
district               0
postalCode             0
localityType           0
bedroomCount           0
netHabitableSurface    0
condition              0
conditionNum           0
bathroomCount          0
showerRoomCount        0
toiletCount            0
hasLift                0
fireplaceExists        0
hasSwimmingPool        0
hasAirConditioning     0
hasGarden              0
gardenSurface          0
hasTerrace             0
terraceSurface         0
price                  0
dtype: int64

In [461]:
#   save new csv
path_to_save = "..\\data\\04_ml_prep_data_drop_duplicate.csv"
df.to_csv(path_to_save, index=True)